# Guiana Chestnut algorithms
This notebook will aim to build several algorithms for fundamental and technical analysis of US stock symbols.

We will be using the [finnhub.io](https://finnhub.io/docs/api) API to download financial data for US stock symbols

### Packages
Please ensure these packages are installed on your local environment via ```pip install -r requirements.txt``` or the corresponding package manager on your OS.

In [28]:
import csv
import numpy as np
import pandas as pd
import os
import datetime

import finnhub as fh
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

%matplotlib inline

### 0. Essential functions

We first define various functions for gathering the data from finnhub.io which we will use for training our algorithms.

Finnhub.io requires a client object to connect to their API. Please register for a free API key if you don't have one.

In [29]:
key = os.environ.get('FINNHUB_API_KEY')
client = fh.Client(api_key=key)
client

In [39]:
def get_ticker_financial_data(symbol, client):
    """
    Builds a pandas dataframe containing the essential financial data of a US stock symbol.
    @params symbol: a registered US stock symbol (ticker symbol).
    @params client: a finnhub.io Client instance initialized with your finnhub.io API key.
    @return: pandas Dataframe
    """

    # Extract company profile
    company_profile_dict = client.company_profile2(symbol=symbol)
    del company_profile_dict['logo'], company_profile_dict['phone'], company_profile_dict['weburl'], company_profile_dict['ticker']
    company_profile = pd.Series(company_profile_dict)

    # Extract company financial metrics
    company_financials_metrics = client.company_basic_financials(symbol=symbol, metric='all')['metric']
    company_financials = pd.Series(company_financials_metrics)    

    data = pd.concat([company_profile, company_financials])
    return pd.DataFrame(data)

def get_ticker_news_data(symbol, client, date_format, days=7):
    """
    Builds a list of news summaries for a US stock symbol.
    @params symbol: a registered US stock symbol (ticker symbol).
    @params client: a finnhub.io Client instance initialized with your finnhub.io API key.
    @return: List of news headlines for further processing
    """

    today       = datetime.datetime.now(datetime.timezone.utc).replace(tzinfo=datetime.timezone.utc).timestamp()
    date_delta  = today - days * 24 * 60 * 60
    start_date  = datetime.datetime.fromtimestamp(date_delta).strftime(date_format)
    today_date  = datetime.datetime.fromtimestamp(today).strftime(date_format)

    news        = client.company_news(symbol='AAPL', _from=start_date, to=today_date)
    news_data   = [n['summary'] for n in news]
    return news_data

### 1. Fundamental analysis on symbol

This section focuses on finding patterns in the financial information of all US symbols (correlations, covariances, etc.).

In [31]:
symbol = 'AAPL'

In [32]:
get_ticker_financial_data(symbol, client)

,0
country,US
currency,USD
exchange,NASDAQ NMS - GLOBAL MARKET
finnhubIndustry,Technology
ipo,1980-12-12
...,...
tangibleBookValuePerShareQuarterly,3.17824
tbvCagr5Y,-17.6806
totalDebt/totalEquityAnnual,238.8104
totalDebt/totalEquityQuarterly,236.9534


### 2. Fundamental - Sentiment analysis on news

We can estimate how positive a news report is using sentiment analysis. This can be useful later on for finding the correlation between the positivity of a piece of news and the price trends of a symbol.

We will be using the spaCy library for performing sentiment analysis on a given piece of news. Run the following command to install the en package which is required for spaCy:

In [33]:
! python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl (12.8 MB)

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


With the english package installed, we can now perform sentiment analyisis.

In [42]:
ticker_news = get_ticker_news_data(symbol, client, date_format="%Y-%m-%d")

In [47]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

for news in ticker_news:
    result = nlp(news)
    news_sentiment = result._.blob.sentiment
    news_polarity, news_subjectivity = news_sentiment.polarity, news_sentiment.subjectivity
    print(news_polarity, news_subjectivity)

0.20833333333333334 0.25
0.24375 0.71875
0.009523809523809526 0.22261904761904763
0.5166666666666666 0.8416666666666666
0.4375 0.9375
0.06818181818181818 0.22727272727272727
0.6 0.44999999999999996
-0.04999999999999996 0.35416666666666663
0.0 0.0
0.0 0.1
0.15 0.0625
0.3333333333333333 0.16666666666666666
0.16944444444444443 0.4263888888888889
0.010000000000000009 0.3511111111111111
0.09166666666666667 0.2722222222222222
0.044444444444444446 0.3277777777777777
0.015000000000000013 0.53
0.5666666666666667 0.6666666666666666
0.16666666666666666 0.45
0.24727272727272726 0.5709090909090909
0.08333333333333333 0.4166666666666667
0.0 0.0
-0.03333333333333335 0.75
0.1814814814814815 0.7222222222222223
0.0 0.4666666666666666
-0.4 0.7
0.5 0.5625
0.2 0.6
0.0 0.0
0.0 0.0
0.2 0.27999999999999997
0.1 0.8125
0.0 0.0
0.25 0.3125
0.075 0.425
0.0 0.0
0.0 0.1
0.13636363636363635 0.5
0.075 0.4944444444444444
0.3055555555555555 0.5166666666666667
-0.11666666666666665 0.11666666666666665
0.0 1.0
0.108333333